<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%205.2%20(2021-02-10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 5.2 (2021-02-10)**
Week 5 continues...
*Reminder, Lab 4 due Friday!*


In [ ]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 5-1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [ ]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)

  fig = plt.figure(figsize=(16,6))

  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlot(): properly formats time domain plot of a signal

In [ ]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [ ]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')
  return fig, plt  

## myPlotFFTPhase

In [ ]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [ ]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

#SNR clarifications


In [ ]:
s11, fs11 = sf.read(ClassPath + 'sounds/Sinatra-11kHz-mono.wav')
s77 = np.zeros(len(s11)*7)
s77[::7] = s11
h_fir = np.hanning(28)
h_fir = h_fir / np.sum(h_fir)
s77f = signal.lfilter(h_fir,1,s77)
s15 = s77f[::5]

s15_target, fs15 = sf.read(ClassPath + 'sounds/Sinatra-15435Hz.wav')

In [ ]:
t = np.arange(len(s15)) / fs15
fig = plt.figure(figsize=(16,4))
plt.plot(t[:100], s15_target[:100], t[:100], s15[2:102])

In [ ]:
sig_pow = s15_target**2
sq_err = (s15_target[:425221] - s15[2:425223])**2

snr1 = np.sum(sig_pow)/np.sum(sq_err)
snr1_dB = 10*np.log10(snr1)
snr1_dB

In [ ]:
eps = np.finfo(float).eps
eps

In [ ]:
snr2 = 10*np.log10(sig_pow + eps) - 10*np.log10(sq_err + eps)
np.mean(snr2)

# Load today's sound files

In [ ]:
s44s, fs44 = sf.read(ClassPath + 'sounds/Sinatra-44kHz.wav')
s44 = np.mean(s44s,axis=1)
ipd.Audio(s44,rate=fs44)

# Ideal low-pass filter (LPF)



In [ ]:
fs = 44100
N = 1024
L = 64
H_ideal = np.append(np.ones(L),np.zeros(N-L))

# Filter needs to be symmetric
H_ideal_sym = np.append(H_ideal, np.flipud(H_ideal))

fig = plt.figure(figsize=(16,4))
f = np.arange(len(H_ideal_sym)) * fs / len(H_ideal_sym)
plt.plot(f, H_ideal_sym)

In [ ]:
f_Hz = np.arange(-N,N)*(fs/2) / N
#f_norm = np.arange(-N,N) / N
#f_rad = np.arange(-N,N)*np.pi / N

fig = plt.figure(figsize=(16,4))
plt.plot(f_Hz, np.fft.fftshift(H_ideal_sym))

plt.xlabel('Frequency (Hz)')
#plt.xlabel('Frequency (normalized)')
#plt.xlabel('Frequency (radians)')

## Take the inverse Fourier Transform

In [ ]:
h_ideal_sym = ...
myPlot( ... )

In [ ]:
h_ideal_shift = ...

n = np.arange(-N,N)
fig = plt.figure(figsize=(16,4))
plt.plot(n, h_ideal_shift)
plt.xlabel('Samples')

# Time limit (window) the infinite ideal function

In [ ]:
M = 128
h_ideal_win = h_ideal_shift[1024-M:1024+M]
plt.plot(h_ideal_win)

In [ ]:
myPlotFFT(h_ideal_win, 4096)

In [ ]:
s44_filt = signal.lfilter(h_ideal_win, 1, s44)
plotSpectrogram(s44, fs)

In [ ]:
plotSpectrogram(s44_filt,fs)

In [ ]:
ipd.Audio(s44_filt,rate=fs)

# Attempting to filter in the frequency domain


In [ ]:
s = s44[1:2048]
S = np.fft.fft(s)
s_inv = np.fft.ifft(S)
fig = plt.figure(figsize=(16,4))
plt.subplot(211)
plt.plot(s)
plt.subplot(212)
plt.plot(s_inv)

In [ ]:
f_size = 2048
n_hop = f_size / 2
N_fft = f_size
fs = 44100
L = 512
H_id = np.append(np.ones(L), np.zeros(f_size-L))

def fftFilter(sig, n_frames=0):
  out = np.zeros(len(sig))

  if n_frames == 0:
    n_frames = int(len(sig)/n_hop) - 1
    print(n_frames)

  for i in range(n_frames):
    n1 = int(n_hop*i)
    n2 = int(n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_f = X_i * H_id
    x_f = np.fft.ifft(X_f)

    out[n1:n2] = np.real(x_f)

  return out

In [ ]:
s44_f2 = fftFilter(s44)
plt.plot(s44_f2)
ipd.Audio(s44_f2,rate=fs)